In [33]:
import os
from scipy.io import loadmat
import numpy as np
from scipy import stats
import pandas as pd


import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    

In [34]:
#Variables et dimesionnalité
nb_AD = 28
nb_MCI = 22
nb_SCI = 22
Freqs= ['ALPHA', 'BETA', 'DELTA', 'THETA']
Seuils = [100, 70, 50, 30, 20, 10]
dataFolder = 'EpEn data/'

In [35]:
#Chargement des matricces des trois types des patients
x_AD = []
for i in range(1, nb_AD+1):
    x = {}
    for freq in Freqs:
        x[freq + '_100'] = loadmat(dataFolder + freq + '/' 
                                    + 'AD' + '/' 
                                    + str(i) + '.mat')[freq]
    x_AD.append(x)

x_MCI = []
for i in range(1, nb_MCI+1):
    x = {}
    for freq in Freqs:
        x[freq + '_100'] = loadmat(dataFolder + freq + '/' 
                                    + 'MCI' + '/' 
                                    + str(i) + '.mat')[freq]
    x_MCI.append(x)

x_SCI= []
for i in range(1, nb_SCI+1):
    x = {}
    for freq in Freqs:
        x[freq + '_100'] = loadmat(dataFolder + freq + '/' 
                                    + 'SCI' + '/' 
                                    + str(i) + '.mat')[freq]
    x_SCI.append(x)

In [36]:
#On étend les dictionaires aux 6 différents niveau de seuil
def get_percentile(x, seuil):
    y = x.copy()
    percentile_limit = np.percentile(x, 100 - seuil)
    y[y<percentile_limit] = 0
    return y


for i in range(nb_AD):
    for freq in Freqs:
        for seuil in Seuils:
            x_AD[i][freq+'_'+str(seuil)] = get_percentile(x_AD[i][str(freq)+'_100'], seuil)
            
for i in range(nb_MCI):
    for freq in Freqs:
        for seuil in Seuils:
            x_MCI[i][freq+'_'+str(seuil)] = get_percentile(x_MCI[i][str(freq)+'_100'], seuil)
            
for i in range(nb_SCI):
    for freq in Freqs:
        for seuil in Seuils:
            x_SCI[i][freq+'_'+str(seuil)] = get_percentile(x_SCI[i][str(freq)+'_100'], seuil)

In [37]:
def nb_connexions_electrode(array):
    connexions = []
    for i in range(array.shape[0]):
        connexion = 0
        for j in range(len(connexions)):
            if array[i, j]>0:
                connexion += 1
        connexions.append(connexion)
    return connexions


def variance_cdf(array):
    variance_list = []
    for i in range(array.shape[0]):
        cdf = np.cumsum(array[i, :])
        variance_list.append(np.var(cdf))
    return variance_list

In [38]:
Var=[]
for freq in Freqs:
    for seuil in Seuils:
        Var.append(freq+'_'+str(seuil))

### Séléction sur la base du nb de conx: 

In [39]:
matrices_AD_nb_conx = {}
for var in Var:
    matrice=[]
    for i in range (len(x_AD)):
        t = nb_connexions_electrode(x_AD[i][var])
        matrice.append(t)
    matrices_AD_nb_conx[var]= matrice
    
matrices_MCI_nb_conx = {}
for var in Var:
    matrice=[]
    for i in range (len(x_MCI)):
        t = nb_connexions_electrode(x_MCI[i][var])
        matrice.append(t)
    matrices_MCI_nb_conx[var]= matrice
    

matrices_SCI_nb_conx = {}
for var in Var:
    matrice=[]
    for i in range (len(x_SCI)):
        t = nb_connexions_electrode(x_SCI[i][var])
        matrice.append(t)
    matrices_SCI_nb_conx[var]= matrice

In [40]:
Elec = ['Elec1', 'Elec2', 'Elec3', 'Elec4', 'Elec5', 'Elec6', 'Elec7', 'Elec8', 'Elec9', 'Elec10', 'Elec11', 'Elec12', 'Elec13', 'Elec14', 'Elec15', 'Elec16', 'Elec17', 'Elec18', 'Elec19', 'Elec20', 'Elec21', 'Elec22', 'Elec23', 'Elec24', 'Elec25', 'Elec26', 'Elec27', 'Elec28', 'Elec29', 'Elec30']

In [41]:
def dataframe_creator(array,names):
    x = pd.DataFrame(array,columns=names)
    return x


### Selection sur la base de la var cfd: 

In [42]:
matrices_AD_Var = {}
for var in Var:
    matrice=[]
    for i in range (len(x_AD)):
        t = variance_cdf(x_AD[i][var])
        matrice.append(t)
    matrices_AD_Var[var]= matrice
    
matrices_MCI_Var = {}
for var in Var:
    matrice=[]
    for i in range (len(x_MCI)):
        t = variance_cdf(x_MCI[i][var])
        matrice.append(t)
    matrices_MCI_Var[var]= matrice
    

matrices_SCI_Var = {}
for var in Var:
    matrice=[]
    for i in range (len(x_SCI)):
        t = variance_cdf(x_SCI[i][var])
        matrice.append(t)
    matrices_SCI_Var[var]= matrice

### Chargement de la base de données  séléctionnées par la combinaison des deux extracteurs 

In [43]:
data_sci_ad = pd.read_csv('datas_enrg/data_sci_ad_app2.csv',sep = ',',index_col=0)
data_ad_mci = pd.read_csv('datas_enrg/data_ad_mci_app2.csv',sep = ',',index_col=0)
data_sci_mci = pd.read_csv('datas_enrg/data_sci_mci_app2.csv',sep = ',',index_col=0)

In [44]:
AD_MCI_selected = data_ad_mci.columns
SCI_MCI_selected = data_sci_mci.columns
SCI_AD_selected = data_sci_ad.columns

In [45]:
def df_ad_mci():    
    data_frame_AD_MCI = pd.DataFrame()
    for i in AD_MCI_selected:
        extr,freq,seuil,elec = i.split('_')
        index = freq + '_' + seuil
        if(extr == 'var'):
            data_AD = dataframe_creator(matrices_AD_Var[index],Elec).add_prefix('var_' + index+'_')
            data_SCI = dataframe_creator(matrices_SCI_Var[index],Elec).add_prefix('var_'+ index+'_')
            data_MCI = dataframe_creator(matrices_MCI_Var[index],Elec).add_prefix('var_'+ index+'_')
        else:
            data_AD = dataframe_creator(matrices_AD_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            data_SCI = dataframe_creator(matrices_SCI_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            data_MCI = dataframe_creator(matrices_MCI_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            
        
        df_tmp = dataframe_creator(pd.concat([data_AD[extr + '_' + index +'_'+ elec],data_SCI[extr + '_' + index+'_'+elec],data_MCI[extr + '_' + index +'_'+ elec]]),[extr + '_' + index+'_'+elec])

        data_frame_AD_MCI = pd.concat([data_frame_AD_MCI,df_tmp],axis = 1)
    return(data_frame_AD_MCI)

#######################################################################################################
def df_sci_mci():    
    data_frame_SCI_MCI = pd.DataFrame()
    for i in SCI_MCI_selected:
        extr,freq,seuil,elec = i.split('_')
        index = freq + '_' + seuil
        if(extr == 'var'):
            data_AD = dataframe_creator(matrices_AD_Var[index],Elec).add_prefix('var_' + index+'_')
            data_SCI = dataframe_creator(matrices_SCI_Var[index],Elec).add_prefix('var_'+ index+'_')
            data_MCI = dataframe_creator(matrices_MCI_Var[index],Elec).add_prefix('var_'+ index+'_')
        else:
            data_AD = dataframe_creator(matrices_AD_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            data_SCI = dataframe_creator(matrices_SCI_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            data_MCI = dataframe_creator(matrices_MCI_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            
        
        df_tmp = dataframe_creator(pd.concat([data_AD[extr + '_' + index +'_'+ elec],data_SCI[extr + '_' + index+'_'+elec],data_MCI[extr + '_' + index +'_'+ elec]]),[extr + '_' + index+'_'+elec])

        data_frame_SCI_MCI = pd.concat([data_frame_SCI_MCI,df_tmp],axis = 1)
    return(data_frame_SCI_MCI)

#######################################################################################################
def df_sci_ad():
    data_frame_SCI_AD = pd.DataFrame()
    for i in SCI_AD_selected:
        extr,freq,seuil,elec = i.split('_')
        index = freq + '_' + seuil
        if(extr == 'var'):
            data_AD = dataframe_creator(matrices_AD_Var[index],Elec).add_prefix('var_' + index+'_')
            data_SCI = dataframe_creator(matrices_SCI_Var[index],Elec).add_prefix('var_'+ index+'_')
            data_MCI = dataframe_creator(matrices_MCI_Var[index],Elec).add_prefix('var_'+ index+'_')
        else:
            data_AD = dataframe_creator(matrices_AD_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            data_SCI = dataframe_creator(matrices_SCI_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            data_MCI = dataframe_creator(matrices_MCI_nb_conx[index],Elec).add_prefix('nbco_'+ index+'_')
            
        
        df_tmp = dataframe_creator(pd.concat([data_AD[extr + '_' + index +'_'+ elec],data_SCI[extr + '_' + index+'_'+elec],data_MCI[extr + '_' + index +'_'+ elec]]),[extr + '_' + index+'_'+elec])

        data_frame_SCI_AD = pd.concat([data_frame_SCI_AD,df_tmp],axis = 1)
    return(data_frame_SCI_AD)

In [46]:
df_sci_ad()

,var_DELTA_20_Elec27,nbco_DELTA_20_Elec27,nbco_ALPHA_50_Elec7,nbco_DELTA_70_Elec27,var_ALPHA_20_Elec11
0,0.279366,4,1,22,4.168060
1,0.292944,4,2,15,0.501223
2,1.297748,7,3,23,1.881494
3,0.056984,1,6,12,2.724715
4,2.240795,6,3,23,2.661411
...,...,...,...,...,...
17,7.828150,13,0,24,0.541508
18,6.417367,11,0,25,1.071973
19,0.344967,4,3,14,5.645392
20,0.405136,3,0,19,0.694949


In [47]:
df_ad_mci()

,var_BETA_50_Elec21,var_DELTA_20_Elec9,nbco_BETA_30_Elec4,nbco_DELTA_20_Elec9,var_BETA_70_Elec16
0,7.481857,8.707088e-02,1,2,10.068019
1,5.916788,5.639033e-01,0,1,3.944881
2,4.810352,1.232595e-32,2,1,11.368997
3,5.641336,0.000000e+00,1,0,10.104764
4,7.917034,8.396366e-01,1,0,8.445762
...,...,...,...,...,...
17,13.458321,3.391840e+00,0,2,16.167311
18,11.404411,9.379430e-01,0,2,16.092545
19,16.383122,6.744095e-01,0,1,16.564530
20,14.138322,1.182684e+01,0,3,13.810771


In [48]:
df_sci_mci()

,nbco_DELTA_70_Elec5,nbco_ALPHA_50_Elec11,nbco_BETA_70_Elec27,var_DELTA_50_Elec5,nbco_ALPHA_10_Elec5
0,3,6,15,2.787759,0
1,4,9,23,18.733543,0
2,3,6,17,10.222861,0
3,2,8,8,8.935979,1
4,3,5,18,2.331296,0
...,...,...,...,...,...
17,1,4,21,0.033631,0
18,0,2,17,0.000000,0
19,4,8,21,5.335175,0
20,4,5,24,18.147874,0


In [49]:
data_frame_kmeans =  pd.concat([df_ad_mci(),df_sci_mci(),df_sci_ad()],axis = 1)

In [50]:
data_frame_kmeans = data_frame_kmeans.reset_index(drop=True)


### Normalisation de cette nouvelle DF 

In [51]:
def centre_and_reduce(X):
    mean = X.mean(axis=0)
    stddev = X.std(axis=0,ddof=1)
    # centered_reduced = centered[:, None] / stddev
    return ((X-mean)/stddev)


data_frame_kmeans = centre_and_reduce(data_frame_kmeans)
data_frame_kmeans.columns

Index(['var_BETA_50_Elec21', 'var_DELTA_20_Elec9', 'nbco_BETA_30_Elec4',
       'nbco_DELTA_20_Elec9', 'var_BETA_70_Elec16', 'nbco_DELTA_70_Elec5',
       'nbco_ALPHA_50_Elec11', 'nbco_BETA_70_Elec27', 'var_DELTA_50_Elec5',
       'nbco_ALPHA_10_Elec5', 'var_DELTA_20_Elec27', 'nbco_DELTA_20_Elec27',
       'nbco_ALPHA_50_Elec7', 'nbco_DELTA_70_Elec27', 'var_ALPHA_20_Elec11'],
      dtype='object')

### Enregistrement de la data frame créee sous format csv  

In [52]:
data_frame_kmeans.to_csv('datas_enrg/data_frame_kmeans.csv',index=True)